### Modules needed to Train the model

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, save_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

#### Base Model [VGG16]

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)  # Adjust the number of units based on the number of classes

In [ ]:
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for layer in base_model.layers[-3:]:
    layer.trainable = True

In [ ]:
# Compile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

#### My Model

In [ ]:
# Create the Sequential model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Define the input shape with the Input layer
    
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),

    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(6, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

##### Preparing Train, Test and Validation Data

In [ ]:
# Data Augmentation and Generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('data/train', target_size=(224, 224), 
                                                    batch_size=64, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory('data/val', target_size=(224, 224), 
                                                        batch_size=64, class_mode='categorical')

In [ ]:
data_dir = 'data/train'

print("Class indices:", train_generator.class_indices)
for cls in train_generator.class_indices:
    cls_dir = os.path.join(data_dir, cls)
    num_images = len(os.listdir(cls_dir))
    print(f"Class {cls} has {num_images} images")

##### Adding weights to counter the bias in the trainable data

In [ ]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

##### Using Early Stopping and ReduceLROnPlateau to dynamically end and adjust learning rate respectively

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6,verbose=1)

In [ ]:
# Train the Model
history = model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[reduce_lr,early_stopping],class_weight = class_weights_dict)

In [ ]:
# Save the Trained Model
# model.save("trained_model.keras")

# Save the model in TensorFlow SavedModel format
# model.save('model.h5')

# Load the model from TensorFlow SavedModel format
model = tf.keras.models.load_model('model.h5')

# model = tf.keras.models.load_model('model_2.h5')

#### Testing The Accuracy of the Trained model

In [ ]:
# Evaluate the Model on Test Data
test_generator = test_datagen.flow_from_directory('data/test', target_size=(224, 224), 
                                                  batch_size=64, class_mode='categorical', shuffle=True)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Get the ground truth labels
test_labels = test_generator.classes
print(test_labels)

In [ ]:
# Predict the classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Print classification report
print(classification_report(test_labels, predicted_classes, target_names=test_generator.class_indices.keys()))

# Print confusion matrix
print(confusion_matrix(test_labels, predicted_classes))